In [1]:
# %load Ads_extraction.py
import pandas as pd
from datetime import datetime
import re
import Utils as utils
import os

# Path to the CSV file containing URLs
csv_file_path = 'kijiji_rental_ads_url.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file_path)
df.head()
# Extract the URLs as a list
url_list = df['URL'].tolist()  # Assuming 'URL' is the column containing the URLs

data = []
counter = 0
start_time = datetime.now()

# Loop through the list of URLs and process each one
for url in url_list:

    soup = utils.get_soup(url)
    if soup:
        # Initialize a dictionary to store the extracted data for this URL
        ad_data = {}

        title_el = soup.find('h1', class_='title-4206718449')
        if title_el is None:
            continue

        # Extract title, price, address, and posting date
        ad_data['Title'] = title_el.text.strip()

        # Extract Price
        price_elem = soup.find(class_='priceWrapper-3915768379')
        price_text = ''
        if price_elem:
            price_text = price_elem.text.strip()
        else:
            backup_price_elem = soup.find(class_='currentPrice-231544276')
            price_text = backup_price_elem.text.strip() if backup_price_elem else None

        ad_data['Price($)'] = ''.join(re.findall(r'\d+', price_text))

        # Extract Address
        address_elem = soup.find(itemprop='address')
        ad_data['Address'] = address_elem.text.strip() if address_elem else None

        # Find the div with class "datePosted"
        date_posted_div = soup.find('div', class_='datePosted-1776470403')

        posted_datetime = ''
        # Extract the datetime attribute from the time tag
        if date_posted_div:
            time_tag = date_posted_div.find('time')
            if time_tag and 'datetime' in time_tag.attrs:
                datetime_str = time_tag['datetime']

                # Parse the datetime string into a datetime object
                posted_datetime = datetime.strptime(datetime_str, "%Y-%m-%dT%H:%M:%S.%fZ")

        ad_data['Date Posted'] = posted_datetime

        # Extract bedrooms, bathrooms, and building type if title_attributes is present
        title_attributes = soup.find(class_='titleAttributes-183069789')
        if title_attributes:
            title_attributes = title_attributes.find_all('li', class_='noLabelAttribute-262950866')
            for attr in title_attributes:
                if 'Bedrooms' in attr.span.text:
                    ad_data['Bedrooms'] = attr.span.text.split(': ')[-1]
                elif 'Bathrooms' in attr.span.text:
                    ad_data['Bathrooms'] = attr.span.text.split(': ')[-1]
                else:
                    ad_data['Building Type'] = attr.span.text

        # Handle case where title_attributes is not present
        else:
            ad_data['Bedrooms'] = None
            ad_data['Bathrooms'] = None
            ad_data['Building Type'] = None

        # Find the container for the "Utilities Included" section
        utilities_included_container = soup.find('h4', string='Utilities Included')

        # Extract the "Utilities Included" section if it exists
        utilities_text = ''
        if utilities_included_container:
            utilities_included_section = utilities_included_container.find_next('ul')
            if utilities_included_section:
                utilities_element = utilities_included_section.find_all('li')
                utilities_text = ','.join(
                    f"{item.get_text(strip=True)}_Yes" if 'Yes' in item.find('svg').get('aria-label',
                                                                                        '') else f"{item.get_text(strip=True)}_No" if 'No' in item.find(
                        'svg').get('aria-label', '') else f"{item.get_text(strip=True)}" for item in utilities_element)

        # Print the combined text with Yes/No indicators
        ad_data['Utilities'] = utilities_text

        # Get and print each section's text
        ad_data['Wi-Fi and More'] = utils.get_section_text(soup, 'Wi-Fi and More', 'h4', 'ul')

        ad_data['Parking Included'] = utils.get_section_text(soup, 'Parking Included', 'dt', 'dd')

        ad_data['Agreement Type'] = utils.get_section_text(soup, 'Agreement Type', 'dt', 'dd')

        ad_data['Move-In Date'] = utils.get_section_text(soup, 'Move-In Date', 'dt', 'dd')

        ad_data['Pet Friendly'] = utils.get_section_text(soup, 'Pet Friendly', 'dt', 'dd')

        ad_data['Size (sqft)'] = utils.get_section_text(soup, 'Size (sqft)', 'dt', 'dd')

        ad_data['Furnished'] = utils.get_section_text(soup, 'Furnished', 'dt', 'dd')

        ad_data['Air Conditioning'] = utils.get_section_text(soup, 'Air Conditioning', 'dt', 'dd')

        ad_data['Personal Outdoor Space'] = utils.get_section_text(soup, 'Personal Outdoor Space', 'h4', 'ul')

        ad_data['Smoking Permitted'] = utils.get_section_text(soup, 'Smoking Permitted', 'dt', 'dd')

        ad_data['Appliances'] = utils.get_multiple_section_text(soup, 'Appliances')

        ad_data['Amenities'] = utils.get_multiple_section_text(soup, 'Amenities')

        # Extract description
        description_elm = soup.select_one('.descriptionContainer-2067035870 p')
        ad_data['Description'] = description_elm.text.strip() if description_elm else None

        # Extract visit counter
        visit_counter_elem = soup.select_one('.visitCounter-204515568 span')
        ad_data['Visit Counter'] = visit_counter_elem.text.strip() if visit_counter_elem else None

        ad_data['url'] = url

        # Append the extracted data to the list
        data.append(ad_data)

        counter += 1
        print("Number ", counter, ' and URL ', url)

# Create a DataFrame from the collected data
df = pd.DataFrame(data)

# Save the DataFrame to a CSV file
df.to_csv(f'kijiji_rental_ads_{counter}.csv', index=False)

# Delete the original CSV file containing URLs
os.remove('kijiji_rental_ads_url.csv')
print("Delete csv kijiji_rental_ads_url.csv")

utils.print_time_info(start_time, datetime.now())


Number  1  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/2-bedrooms-apartment-for-rent-245-howland-avenue/1683272085
Number  2  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/davisville-village-apartments-2-bdrm-available-at-141-davisvil/1686716361
Number  3  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/17-raglan-raglan-gardens-1-bedroom-1-bathroom-apartment-for-r/1675567882
Number  4  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/open-house-gorgeous-newly-built-suites-ready-for-occupancy/1682801754
Number  5  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/newly-designed-1-bdrm-for-rent-woodbine-gerrard-east/1686542799
Number  6  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/brand-new-2-bedroom-rental-in-north-york-york-mills-rd-dvp/1679403688
Number  7  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/live-on-the-main-floor-of-a-modern-house-in

Number  61  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/2-spacious-bedrooms-1-bath-basement-for-rent/1688106810
Number  62  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/spacious-renovated-one-bedroom/1688104141
Number  63  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/1-bedroom-apartment-for-rent-on-the-ttc-subway/1688106611
Number  64  and URL  https://www.kijiji.ca/v-room-rental-roommate/city-of-toronto/apt-sharing-boys/1688106257
Number  65  and URL  https://www.kijiji.ca/v-room-rental-roommate/city-of-toronto/dt-new-condo-yonge-and-dundas-1bedroom-bathroom-available-now/1688106099
Number  66  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/laneway-suite-furnished-1-bed-1-bath-steps-to-christie-n-bloor/1688105932
Number  67  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/2-bed-2-bath-furnished-condo-available-for-rent/1688105681
Number  68  and URL  https://www.kijiji.ca/v-short-

Number  120  and URL  https://www.kijiji.ca/v-room-rental-roommate/city-of-toronto/private-room-with-private-bathroom-and-parking/1688103036
Number  121  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/2-bdrm-large-sunroom-house-huge-yard-free-wifi-parking-curtains/1688103019
Number  122  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/dupont-symington-one-bedroom-basement-apartment/1688102960
Number  123  and URL  https://www.kijiji.ca/v-room-rental-roommate/city-of-toronto/sheppard-bayview-north-york-master-bed-room-to-rent-2min-to-ttc/1688102834
Number  124  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/forest-hill-residences-3-bedroom-apartment-for-rent/1687441956
Number  125  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/la-salle-towers-2-bedroom-apartment-for-rent/1687443162
Number  126  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/basement-for-rent-in-morningside-and-mcnicoll/16

Number  179  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/the-danforth-townhome-for-rent-35-41-danforth-road/1687103826
Number  180  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/the-kew-townhome-for-rent-35-41-danforth-road/1687103827
Number  181  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/the-main-townhome-for-rent-35-41-danforth-road/1684189158
Number  182  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/the-oakridge-townhome-for-rent-35-41-danforth-road/1685268682
Number  183  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/the-bluffs-townhome-for-rent-35-41-danforth-road/1687103829
Number  184  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/the-balmy-townhome-for-rent-35-41-danforth-road/1687103830
Number  185  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/prince-arthur-2-bedroom-apartment-for-rent/1687348723
Number  186  and URL  https:

Number  237  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/davisville-village-apartments-2-bdrm-available-at-141-davisvil/1686716361
Number  238  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/luxury-1-room-in-2-2-apartment-near-queen-w/1685691301
Number  239  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/1-bedroom-apartment-for-rent-570-birchmount-road/1684189178
Number  240  and URL  https://www.kijiji.ca/v-commercial-office-space/city-of-toronto/room-rentals-available-for-health-beauty-professionals-north/1685059577
Number  241  and URL  https://www.kijiji.ca/v-room-rental-roommate/city-of-toronto/emerald-city-condo-donmills-station-1-bed-2-bed-for-rent/1688098786
Number  242  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/studio-apartment-for-rent-1475-birchmount-road/1684189037
Number  243  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/2-bedroom-apartment-for-rent-1265-birchmount

...

Number  2673  and URL  https://www.kijiji.ca/v-room-rental-roommate/city-of-toronto/basement-for-rent/1687861135
Number  2674  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/renovated-two-bedroom-basement-for-rent-near-kennedy-danforth/1687861126
Number  2675  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/3-beds-den-and-3-baths-townhouse/1685733989
Number  2676  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/1-bedroom-apartment-in-the-annex-for-rent-2-min-from-spadina-stn/1685317097
Number  2677  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/fully-furnished-private-bedroom/1687860899
Number  2678  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/main-floor-annex-house-c-backyard-and-walk-to-dupont-subway/1687860941
Number  2679  and URL  https://www.kijiji.ca/v-room-rental-roommate/city-of-toronto/sharing-accommodation-available/1687860924
Number  2680  and URL  https://www.kijiji.ca/v-s

Number  2735  and URL  https://www.kijiji.ca/v-room-rental-roommate/city-of-toronto/1-furnished-bed-in-3rd-floor-900/1687852527
Number  2736  and URL  https://www.kijiji.ca/v-room-rental-roommate/city-of-toronto/3rd-floor-master-bedroom-for-single-female-professional/1687852465
Number  2737  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/3-bedroom-house-for-rent-in-scarborough/1687852043
Number  2738  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/north-east-scarborough-basement-for-rent/1687852041
Number  2739  and URL  https://www.kijiji.ca/v-room-rental-roommate/city-of-toronto/sharing-room-available-for-rent-in-scarborough/1687851922
Number  2740  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/704-75-portland-street-toronto-ontario-m5v-2m9-704-75-portla/1687850943
Number  2741  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/newly-built-2bd-den-basement-apartment-for-rent-in-scarborough/1685840885
Number

Number  2795  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/392-sherbourne-street-sherwood-apartments-apartment-for-rent/1684052966
Number  2796  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/2-bedroom-apartment-north-york-don-valley-parkway-brookbanks-dr/1666459486
Number  2797  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/3-bedroom-apartment-north-york-don-valley-parkway-brookbanks-dr/1667146309
Number  2798  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/two-bedroom-apartment-for-rent/1687841336
Number  2799  and URL  https://www.kijiji.ca/v-room-rental-roommate/city-of-toronto/sharing-room-for-rent/1687841312
Number  2800  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/three-bedroom-house-for-rent-for-student-working-people/1687840081
Number  2801  and URL  https://www.kijiji.ca/v-short-term-rental/city-of-toronto/room-available-for-rent-april-1/1675830353
Number  2802  and URL  

Number  2856  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/1-bedroom-apartment-for-rent-north-york/1539126645
Number  2857  and URL  https://www.kijiji.ca/v-room-rental-roommate/city-of-toronto/all-inclusive-private-furnished-room-north-york-april-1st/1687830808
Number  2858  and URL  https://www.kijiji.ca/v-room-rental-roommate/city-of-toronto/large-sunny-room-in-2-bed-1bath-in-junction/1687085405
Number  2859  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/basement-apartment-for-rent/1687830647
Number  2860  and URL  https://www.kijiji.ca/v-room-rental-roommate/city-of-toronto/furnished-room-for-rent-in-north-york/1687830498
Number  2861  and URL  https://www.kijiji.ca/v-room-rental-roommate/city-of-toronto/private-room-for-rent/1687830488
Number  2862  and URL  https://www.kijiji.ca/v-room-rental-roommate/city-of-toronto/room-for-rent/1687830257
Number  2863  and URL  https://www.kijiji.ca/v-room-rental-roommate/city-of-toronto/living-room-s

Number  2916  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/three-bedrooms-main-floor-bungalow-for-rent/1687820273
Number  2917  and URL  https://www.kijiji.ca/v-room-rental-roommate/city-of-toronto/spacious-master-bedroom-for-rent-from-1-may-2024/1687819996
Number  2918  and URL  https://www.kijiji.ca/v-room-rental-roommate/city-of-toronto/york-university-room/1687819963
Number  2919  and URL  https://www.kijiji.ca/v-apartments-condos/city-of-toronto/1-bed-1-bath-in-brand-new-condo-for-rent/1687819620
Number  2920  and URL  https://www.kijiji.ca/v-room-rental-roommate/city-of-toronto/room-for-rent/1687819590
Number  2921  and URL  https://www.kijiji.ca/v-room-rental-roommate/city-of-toronto/urgent-mccowan-ellesmere-room-rental-parking-backyard-patio/1687819333
Number  2922  and URL  https://www.kijiji.ca/v-room-rental-roommate/city-of-toronto/a-big-upper-level-private-room-for-a-student-or-man-has-a-job/1687819006
Number  2923  and URL  https://www.kijiji.ca/v-apa